In [ ]:
# Unable to short spot 

In [1]:
import json
import requests
import websockets
from requests import Session
import pandas as pd
import time
import hashlib
import hmac
import base64

# For Websocket

In [2]:
url = 'https://api.kucoin.com/api/v1/bullet-public'

In [3]:
x = requests.post(url)

In [4]:
cjson = json.loads(x.text)

In [5]:
endpoint = cjson['data']['instanceServers'][0]['endpoint']
token = cjson['data']['token']

In [6]:
multi = {"id": "1Jpg30DEdU", "type": "openTunnel", "newTunnelId": "bt1", "response": False}
# command used to open tunnel

In [7]:
multi_spotEth = {"id": "1JpoPamgFM",
             "type": "subscribe",
             "topic": "/market/ticker:ETH-USDT",
             "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

In [8]:
multi_futureEth = {"id": 1545910660740,                          
                "type": "subscribe",
                "topic": "/contractMarket/tickerV2:ETHUSDTM",
                "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

# Trade Code

#### NOTE: Can only take advantage of Longing Spot and Shorting Future

In [9]:
import sys
sys.path.append('..')
import classified

# for future api
fapi_passphrase = classified.FPASSPHRASE
fapi_key = classified.FKEY
fapi_secret = classified.FSECRET
# for spot api
api_passphrase = classified.SPASSPHRASE
api_key = classified.SKEY
api_secret = classified.SSECRET

session = Session()

In [10]:
def auth(method, endpoint):
    now = int(time.time() * 1000)
    str_to_sign = str(now) + method.upper() + endpoint
    signature = base64.b64encode(
        hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
        "KC-API-SIGN": signature,
        "KC-API-TIMESTAMP": str(now),
        "KC-API-KEY": api_key,
        "KC-API-PASSPHRASE": passphrase,
        "KC-API-KEY-VERSION": '2',
        "Content-Type": "application/json"
    }
    return headers

In [11]:
class executeTrade:
    
    def spotOrder(self, side, symbol, price, size):
        url = 'https://api.kucoin.com/api/v1/orders'
        now = int(time.time() * 1000)
        data = {"clientOid": "01", "side": side, "symbol": symbol, "price": price,"size": size}
        data_json = json.dumps(data)
        str_to_sign = str(now) + 'POST' + '/api/v1/orders' + data_json
        signature = base64.b64encode(
            hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
        passphrase = base64.b64encode(
            hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
        headers = {
            "KC-API-SIGN": signature,
            "KC-API-TIMESTAMP": str(now),
            "KC-API-KEY": api_key,
            "KC-API-PASSPHRASE": passphrase,
            "KC-API-KEY-VERSION": '2',
            "Content-Type": "application/json" # specifying content type or using json=data in request
        }
        response = session.post(url, headers=headers, data=data_json)
        print(response.status_code)
        print(response.json())
        
    def futureOrder(self, side, symbol, leverage, price, size):
        url = 'https://api-futures.kucoin.com/api/v1/orders'
        now = int(time.time() * 1000)
        data = {"clientOid": "02", "side": side, "symbol": symbol, "leverage": leverage, "price": price,"size": size}
        data_json = json.dumps(data)
        str_to_sign = str(now) + 'POST' + '/api/v1/orders' + data_json
        signature = base64.b64encode(
            hmac.new(fapi_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
        passphrase = base64.b64encode(
            hmac.new(fapi_secret.encode('utf-8'), fapi_passphrase.encode('utf-8'), hashlib.sha256).digest())
        headers = {
            "KC-API-SIGN": signature,
            "KC-API-TIMESTAMP": str(now),
            "KC-API-KEY": fapi_key,
            "KC-API-PASSPHRASE": passphrase,
            "KC-API-KEY-VERSION": '2',
            "Content-Type": "application/json" # specifying content type or using json=data in request
        }
        response = session.post(url, headers=headers, data=data_json)
        print(response.status_code)
        print(response.json())
        
trade = executeTrade()

In [40]:
class openClosePostion:
    
#     def arbOpen(self, spotPrice, futurePrice):
#         trade.spotOrder("buy", "ETH-USDT", spotPrice, "0.01")
#         trade.futureOrder("sell", "ETHUSDTM", "1", futurePrice, "1")
#         #trade.marekt("buy/sell", "tradingPair", "leverage", "limitPrice", "size")
    
    def arbOpen(self):
        trade.spotOrder("buy", "ETH-USDT", "500", "0.01")
        trade.futureOrder("sell", "ETHUSDTM", "1", "986", "1")
        #trade.marekt("buy/sell", "tradingPair", "leverage", "limitPrice", "size")
        
    def arbClose(self):
        trade.spotOrder("sell", "ETH-USDT", spotPrice, "0.01")
        trade.futureOrder("buy", "ETHUSDTM", "1", futurePrice, "1")
        #trade.marekt("buy/sell", "tradingPair", "leverage", "limitPrice", "size")
        
arb = openClosePostion()

# Websocket Code

In [44]:
async with websockets.connect(endpoint+"?token="+token) as conn:
    await conn.send(json.dumps(multi))  # initial open tunel command
    res = await conn.recv()
    
    await conn.send(json.dumps(multi_spotEth)) # send tunnel subscription
    res = await conn.recv()
    await conn.send(json.dumps(multi_futureEth))  # send tunnel subscription
    res = await conn.recv()
    
    counter = 0
    spotPrice = 0
    futurePrice = 0
    while True:
        res = await conn.recv()
        vals = json.loads(res)

        if (vals['topic']) == '/contractMarket/tickerV2:ETHUSDTM':
#             print(f"Future Price is: {vals['data']['bestAskPrice']}")
            futurePrice = float(vals['data']['bestAskPrice'])

        elif (vals['topic']) == '/market/ticker:ETH-USDT':
#             print(f"Spot Price is: {vals['data']['bestAsk']}")
            spotPrice = float(vals['data']['bestAsk'])
    
        print(spotPrice - futurePrice)
        
        # OPEN POSTION
        if spotPrice - futurePrice > 1 and futurePrice != 0 and spotPrice != 0:
            print('Long Future and Short Spot')
            
        elif futurePrice - spotPrice > 1 and futurePrice != 0 and spotPrice != 0:
            print('Long Spot and Short Future')
#             arb.arbOpen(spotPrice, futurePrice)
            
        # CLOSE POSTION
#         if spotPrice - futurePrice < 0.1 and futurePrice != 0 and spotPrice != 0:
#             print('Long Future and Short Spot')
            
#         elif futurePrice - spotPrice < 0.1 and futurePrice != 0 and spotPrice != 0:
#             print('Long Spot and Short Future')
# #             arb.arbOpen(spotPrice, futurePrice)
            
        counter += 1
    eth_tunnel = {"id": "1JpoPamgFM", "type": "closeTunnel", "tunnelId": "bt1", "response": False}            
    btc_tunnel = {"id": "1JpoPamgFN", "type": "closeTunnel", "tunnelId": "bt1", "response": False}
    await conn.send(json.dumps(eth_tunnel))
    await conn.send(json.dumps(btc_tunnel))
    await conn.close()

1031.53
0.7300000000000182
0.9600000000000364
0.9600000000000364
0.9600000000000364
0.9600000000000364
1.0299999999999727
Long Future and Short Spot
1.0299999999999727
Long Future and Short Spot
1.0699999999999363
Long Future and Short Spot
1.0199999999999818
Long Future and Short Spot
1.0199999999999818
Long Future and Short Spot
1.0199999999999818
Long Future and Short Spot
1.0199999999999818
Long Future and Short Spot
1.0199999999999818
Long Future and Short Spot
1.1000000000001364
Long Future and Short Spot
1.0499999999999545
Long Future and Short Spot
1.0499999999999545
Long Future and Short Spot
1.0499999999999545
Long Future and Short Spot
1.0499999999999545
Long Future and Short Spot
1.0499999999999545
Long Future and Short Spot
1.0499999999999545
Long Future and Short Spot
1.0499999999999545
Long Future and Short Spot
1.0
1.0
1.0
1.0
1.0
1.0499999999999545
Long Future and Short Spot
1.0
1.0
1.0299999999999727
Long Future and Short Spot
1.0299999999999727
Long Future and Short 

0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.7300000000000182
0.8499999999

CancelledError: 

In [38]:
arb.arbOpen()

200
{'code': '200000', 'data': {'orderId': '62bdd3517c7ecc0001cf8ada'}}
200
{'code': '200000', 'data': {'orderId': '62bdd351ca20da0001a02c5a'}}
